In [2]:
import tensorflow as tf
from tensorflow.keras import Model
import numpy as np

# Util functions Test

# acc

In [3]:
def accuracy(output, target):    
    thresh = 0.5
    res = tf.logical_and((output > thresh),(target > thresh))
    return tf.divide(tf.reduce_sum(tf.dtypes.cast(res, tf.int32)), tf.size(output), name="accuracy")

# iou

In [4]:
def iou(output, target):
    """Return IoU, intersection, union

    Args:
        output (tensor): logits/output tensor
        target (tensor): label/target tensor

    Returns:
        (tensor, tensor, tensor): iou, intersection, union
    """
    output = tf.squeeze((output > 0.5))
    target = tf.squeeze((target > 0.5))
    smoothy = 1e-6
    
    intersection = tf.reduce_sum(tf.dtypes.cast(tf.logical_and(output, target), tf.float32), name="intersection")
    union = tf.reduce_sum(tf.dtypes.cast(tf.logical_or(output, target), tf.float32), name="union")

    iou = tf.divide((intersection), (union + smoothy), name="iou")

    return iou, intersection, union

## Tests

In [5]:
from model_search import Network
criterion = tf.losses.sigmoid_cross_entropy
model = Network(3, 3, criterion)

W0917 14:15:16.306579 139948122888000 deprecation.py:506] From /home/mythrex/anaconda3/envs/dl/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0917 14:15:16.309554 139948122888000 deprecation_wrapper.py:119] From /home/mythrex/Coding/Darts-Unet/cnn/operations.py:61: The name tf.layers.Conv2D is deprecated. Please use tf.compat.v1.layers.Conv2D instead.

W0917 14:15:16.310364 139948122888000 deprecation_wrapper.py:119] From /home/mythrex/Coding/Darts-Unet/cnn/operations.py:67: The name tf.layers.BatchNormalization is deprecated. Please use tf.compat.v1.layers.BatchNormalization instead.

W0917 14:15:16.312270 139948122888000 deprecation_wrapper.py:119] From /home/mythrex/Coding/Darts-Unet/cnn/operations.py:7: The na

In [6]:
np_ds_train = (np.random.randint(0, 256, (20, 16, 16, 3)).astype(np.float32), np.random.randint(0, 2, (20, 16, 16, 1)).astype(np.float32))
np_ds_valid = (np.random.randint(0, 256, (20, 16, 16, 3)).astype(np.float32), np.random.randint(0, 2, (20, 16, 16, 1)).astype(np.float32))
ds_train = tf.data.Dataset.from_tensor_slices(np_ds_train).batch(4)
ds_valid = tf.data.Dataset.from_tensor_slices(np_ds_valid).batch(4)

In [7]:
train_it = ds_train.make_one_shot_iterator()
valid_it = ds_valid.make_one_shot_iterator()

W0917 14:15:16.423480 139948122888000 deprecation.py:323] From <ipython-input-7-015b8e901fc6>:1: DatasetV1.make_one_shot_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


In [8]:
with tf.Session() as sess:
    x_train, y_train = sess.run(train_it.get_next())
    sess.run(tf.global_variables_initializer())
    tf.initialize_all_variables().run()
    net_out = model(x_train)
    logits = sess.run(net_out)
    acc = accuracy(output=logits, target=y_train)
    iou = iou(logits, y_train)
    res_acc = sess.run(acc)
    res_iou = sess.run(iou)

W0917 14:15:16.531543 139948122888000 deprecation.py:323] From /home/mythrex/anaconda3/envs/dl/lib/python3.7/site-packages/tensorflow/python/util/tf_should_use.py:193: initialize_all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
W0917 14:15:17.804575 139948122888000 deprecation.py:323] From /home/mythrex/anaconda3/envs/dl/lib/python3.7/site-packages/tensorflow/python/ops/array_ops.py:1354: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


FailedPreconditionError: Error while reading resource variable network/sequential/batch_normalization/beta from Container: localhost. This could mean that the variable was uninitialized. Not found: Resource localhost/network/sequential/batch_normalization/beta/N10tensorflow3VarE does not exist.
	 [[node network/sequential/batch_normalization/cond/ReadVariableOp_3 (defined at /tmp/tmpy9xyx9tc.py:8) ]]

Original stack trace for 'network/sequential/batch_normalization/cond/ReadVariableOp_3':
  File "/home/mythrex/anaconda3/envs/dl/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/mythrex/anaconda3/envs/dl/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/mythrex/anaconda3/envs/dl/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/mythrex/anaconda3/envs/dl/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_in

In [ ]:
res_acc

In [ ]:
res_iou